In [1]:
import numpy as np
import cv2
import math

In [2]:
video_location = "LaneDetectionTestVideo01.mp4"
lane_video = cv2.VideoCapture(video_location)
ok, frame = lane_video.read()

In [3]:
# White colour range (RGB)
lower_w = np.array([100,100,200], dtype=np.uint8)
upper_w = np.array([255,255,255], dtype=np.uint8)

# Yellow colour range (RGB)
lower_y = np.array([140, 130, 70], dtype=np.uint8)
upper_y = np.array([250, 240, 120], dtype=np.uint8)

In [4]:
# Creating the lane mask using the specs of the first frame in the video
width = frame.shape[1]
height = frame.shape[0]

# Region of interest is based on the video/image, in this case this is where the lane typically was
width_l = width // 7
width_r = width - width // 7

height_b = height
height_t = height // 11 * 8

# Triangle polygon
tri_poly = np.array([
                        [
                            (width_l, height_b),
                            (width_r, height_b), 
                            ((width_r + width_l) // 2, height_t)
                        ]
                    ])

lane_mask = np.zeros((frame.shape[0], frame.shape[1]))
lane_mask = np.uint8(lane_mask)
cv2.fillPoly(lane_mask, tri_poly, 255)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [5]:
print("Press q to quit.\n");

# Reading the frames and displaying them in order (as a video)
while (ok):
    # Filtering the frame using a white and yellow mask
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    white_mask = cv2.inRange(frame_rgb, lower_w, upper_w)
    yellow_mask = cv2.inRange(frame_rgb, lower_y, upper_y)
    both_masks = cv2.bitwise_or(white_mask, yellow_mask)
    frame_filtered = cv2.bitwise_and(frame, frame, mask = both_masks)
    
    # Converting the filtered image to gray
    frame_gray = cv2.cvtColor(frame_filtered, cv2.COLOR_RGB2GRAY)

    # Performing gaussian blur on the gray image
    frame_blur = cv2.GaussianBlur(frame_gray, (5, 5), 0)
    frame_blur_u8 = np.uint8(frame_blur)
    
    # Performing canny edge detection the blurred image
    frame_edges = cv2.Canny(frame_blur_u8, 100, 300)  
    
    # Obtaining only the part of the image that we are interested in (the lane)
    frame_lane = cv2.bitwise_and(frame_edges, lane_mask)

    # Performing hough transform and obtaining the hough lines of frame_lane
    lines = cv2.HoughLinesP(frame_lane, 2, np.pi / 180, 50, lines = np.array([]), minLineLength = 25, maxLineGap = 40)
    
    if lines is not None:
        for line in lines:
            # Getting coordinates from line to draw it
            x1, y1, x2, y2 = line[0]
            angle = math.atan2(y2 - y1, x2 - x1) * 180.0 / np.pi;
            
            # Ignoring lines that are close to horizontal
            if(angle < -15 or angle > 15):
                # Drawing a line in frame_gray based on arguments
                cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)
        
    cv2.imshow(video_location, frame)

    if (cv2.waitKey(1) & 0xFF == ord('q')):
        break

    ok, frame = lane_video.read() # return value of ok, and frame
        
# The following frees up resources and closes all windows
lane_video.release()
cv2.destroyAllWindows()

Press q to quit.

